### Package Installation

In [ ]:
!pip install imageai
!pip install deeplabcut
!pip install tensorflow-object-detection-api

  Using cached numpy-1.19.3-cp37-cp37m-manylinux2010_x86_64.whl (14.9 MB)
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached Pillow-7.0.0-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.0.1
    Uninstalling Pillow-9.0.1:
      Successfully uninstalled Pillow-9.0.1
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.4.3 which is incompatible.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.3 which is inco

  Using cached Pillow-9.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.3
    Uninstalling numpy-1.19.3:
      Successfully uninstalled numpy-1.19.3
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageai 2.1.6 requires keras==2.4.3, but you have keras 2.8.0 which is incompatible.
imageai 2.1.6 requires numpy==1.19.3, but

In [ ]:
#base packages
import os
from os import path
import shutil
import numpy as np
from keras.utils import np_utils
import pandas as pd
from PIL import Image
from imageai.Detection import ObjectDetection
import matplotlib.pyplot as plt
import cv2
from operator import itemgetter
import tqdm

#training packages
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPool2D, Flatten, BatchNormalization, Activation, SeparableConv2D, MaxPooling2D, add, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.metrics import Precision, Recall

print("Tensorflow Version: "+tf.__version__)

Tensorflow Version: 2.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
working_path = '/content/drive/MyDrive/Image data'
os.chdir(working_path)

FileNotFoundError: ignored

In [ ]:
pathData = '/content/drive/MyDrive/Image data/training'
newPath = '/content/drive/MyDrive/Image data/labeled-data'
modelPath = '/content/drive/MyDrive/Image data/models/saved_models/resnet50_coco_best_v2.0.1.h5'
padding = 0.1

### Image Cleaning

In [ ]:
# set up dog detection model
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(modelPath)
detector.loadModel()
custom = detector.CustomObjects(dog=True)

In [ ]:
# loop over all images
for folder in os.listdir(pathData):
    if folder != '.comments':
        print('----------------------------------------')
        print(folder + ":")
        path2 = pathData + '/' + folder     # path to emotion folder
        counter = 1     # image counter
        for filename in sorted(os.listdir(path2)):
            if filename != '.comments':
                pathImage = path2 + '/' + filename
                list = filename.split('.')
                pathNewImage = newPath + '/' + folder + str('{:03d}'.format(counter)) + '.jpg'
                if not path.exists(pathNewImage):
                    # detect dog
                    dimage, detections = detector.detectCustomObjectsFromImage(custom_objects=custom, input_image=pathImage, output_type='array')

                    if detections:
                        image = cv2.imread(pathImage)
                        height, width, channels = image.shape

                        dog = (sorted(detections, key=lambda dog: dog['percentage_probability'], reverse=True))[0]
                        x1 = max(0, int(round(dog['box_points'][0] * (1 - padding))))
                        y1 = max(0, int(round(dog['box_points'][1] * (1 - padding))))
                        x2 = min(width, int(round(dog['box_points'][2] * (1 + padding))))
                        y2 = min(height, int(round(dog['box_points'][3] * (1 + padding))))

                        image = image[y1:y2, x1:x2]
                        if image.size:
                            cv2.imwrite(pathNewImage, image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
                            print(pathImage + ' --> ' + pathNewImage)
                            counter = counter + 1
                        else: print('Empty Image!')
                    else: print('No Dog: ' + pathImage)

### Posing

In [ ]:
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.core import predict
from skimage.util import img_as_ubyte
from skimage.io import imread
from skimage.transform import resize
import tensorflow as tf
import os
import cv2
import numpy as np
import tensorflow.compat.v1 as tfc
from object_detection.utils import label_map_util
import pandas as pd

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
sourcePath = '/content/drive/MyDrive/Image data/labeled_data'
targetPath = '/content/drive/MyDrive/Image data/keypoints'

resizeWidth = 300                   # resize width for model prediction
probability = 0.05

probabilityAP = 0.005                # probability for keypoints only in animal pose data set
animalPose = [20, 21, 22, 23]       # animal pose data set points

probabilitySE = 0.05
stanfordExtra = [13, 17, 18, 19]

printNumbers = [0, 3, 6, 9, 12, 13, 14, 15, 16, 17, 18, 19 ,22]     # keypoints to be displayed
printNumbers = [22]     # keypoints to be displayed

data = []
bodyparts = ['L_F_Paw','L_F_Paw','L_F_Wrist','L_F_Wrist','L_F_Elbow','L_F_Elbow',
		 'L_B_Paw','L_B_Paw','L_B_Wrist','L_B_Wrist','L_B_Elbow','L_B_Elbow',
		 'R_F_Paw','R_F_Paw','R_F_Wrist','R_F_Wrist','R_F_Elbow','R_F_Elbow',
		 'R_B_Paw','R_B_Paw','R_B_Wrist','R_B_Wrist','R_B_Elbow','R_B_Elbow',
		 'Tailset','Tailset','Tailtip', 'Tailtip',
		 'L_Base_Ear','L_Base_Ear','R_Base_Ear','R_Base_Ear',
		 'Nose','Nose','Chin','Chin',
		 'L_Tip_Ear','L_Tip_Ear','R_Tip_Ear','R_Tip_Ear',
		 'L_Eye','L_Eye','R_Eye','R_Eye',
		 'Withers','Whithers','Throat','Throat']

outputPath = '/content/drive/MyDrive/Image data/keypoints.csv'

# test settings
test = False
testNumber = 20

###Prepare DeepLabCut

/content/drive/MyDrive/Image data/models/DogPoints-KF-2020-09-20/dlc-models/iteration-0/DogPointsSep20-trainset95shuffle1/test/pose_cfg.yaml

In [ ]:
path_test_config = '/content/drive/MyDrive/Image data/models/DogPoints-KF-2020-09-20/dlc-models/iteration-0/DogPointsSep20-trainset95shuffle1/test/pose_cfg.yaml'
dlc_cfg = load_config(str(path_test_config))
sess, inputs, outputs = predict.setup_pose_prediction(dlc_cfg)      # sess, inputs, outputs

###Prepare Images

In [ ]:
!pip install pillow_heif

In [ ]:
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()

In [ ]:
working_path = '/content/drive/MyDrive/Image data'
os.chdir(working_path)

In [ ]:
os.getcwd()

In [ ]:
start_path = working_path + '/training/'

In [ ]:
import os
import glob

files = glob.glob(sourcePath + '/*')
for f in files:
    os.remove(f)

In [ ]:
for folder in os.listdir(start_path):
  print(folder)
  for count, f in enumerate(os.listdir(start_path + folder)):
    num = count + 1
    f_name, f_ext = os.path.splitext(f)
    src_dir = start_path + folder
    src_file = os.path.join(src_dir, f)
    dst_file = os.path.join(sourcePath, f)
    shutil.copy(src_file, sourcePath)
    new_name = folder + '_' + str(num) + '.jpg'
    new_file_name = os.path.join(sourcePath, new_name)
    os.chdir(sourcePath)
    os.rename(f, new_name)
    im1 = Image.open(new_name)
    im1 = im1.convert('RGB')
    im1.save(new_name)
    os.chdir(working_path)

### Predict Keypoints

In [ ]:
print('########################################################################')
print('DeepLabCut Prediction')

# deeplabcut detection
testCounter = 0
# probCounter = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0, 18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for filename in sorted(os.listdir(sourcePath)):
    if filename != '.DS_Store' and ((test and testCounter < testNumber) or test == False):
        pathImage = sourcePath + '/' + filename
        pathImageNew = targetPath + '/' + filename
        print(pathImage + ' --> ' + pathImageNew)

        # prepare image
        image = imread(pathImage)  # read image
        print(image.shape)
        height = image.shape[0]
        width = image.shape[1]
        resizeHeight = int(round(height * resizeWidth / width))
        image = resize(image, (resizeHeight, resizeWidth), anti_aliasing=False)
        frame = img_as_ubyte(image)

        # apply own DLC detector
        pose = predict.getpose(frame, dlc_cfg, sess, inputs, outputs)  # detect keypoints
        poseResized = []
        scaleFactor = width / resizeWidth

        image = cv2.imread(pathImage)
        counterPoint = 0
        coordinates = []
        #coordinates.append(filename)
        coordinates.append(pathImageNew)

        # determine emotion
        #if 'Anger' in filename:
           # coordinates.append('anger')
       # elif 'Fear' in filename:
          #  coordinates.append('fear')
       # elif 'Happiness' in filename:
         #   coordinates.append('happiness')
        #elif 'Relaxation' in filename:
           # coordinates.append('relaxation')

        for keypoint in pose:
            good = False
            # probCounter[counterPoint] = probCounter[counterPoint] + keypoint[2]
            if counterPoint in animalPose and not keypoint[2] < probabilityAP:
                good = True
            elif counterPoint in stanfordExtra and not keypoint[2] < probabilitySE:
                good = True
            elif counterPoint not in animalPose and counterPoint not in stanfordExtra and not keypoint[2] < probability:
                good = True

            if good:
                x = int(round(keypoint[0]*scaleFactor))
                y = int(round(keypoint[1]*scaleFactor))
                #cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(image, str(counterPoint), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                coordinates.append(x)
                coordinates.append(y)
            else:
                coordinates.append(-1)
                coordinates.append(-1)

            counterPoint += 1

        cv2.imwrite(pathImageNew, image)
        data.append(coordinates)
        testCounter += 1

df = pd.DataFrame(data)
df.to_csv(outputPath, index=False, header=None)

In [ ]:
sourcePath = '/content/drive/MyDrive/Image data/keypoints.csv'
outputPath = '/content/drive/MyDrive/Image data/keypoints_scaled.csv'

coordinates = pd.read_csv(sourcePath, header=None)

#####################################################################
#####################################################################


# check if coordinate exists
def coordinateNotNull(point):
    if point == [-1,-1]: return False
    return True

# get the first filled coordinate of a list
# None = no filled coordinate in list
def getFilledCoordinate(pointList):
    for point in pointList:
        if coordinateNotNull(point): return point
    return None

# mirror image horizontally
def mirrorImage(width, point):
    if coordinateNotNull(point): return [width - point[0], point[1]]
    else: return point

# scale coordinate
def scaleImage(width, height, point):
    if coordinateNotNull(point): return [point[0]/width, point[1]/height]
    else: return point

#####################################################################
#####################################################################

data = []
# iterate over images
for row in range(0, len(coordinates.index)):
    # create list for each dog keypoint
    emotion = coordinates[0][row]
    L_F_Paw = [coordinates[1][row], coordinates[2][row]]
    L_F_Wrist = [coordinates[3][row], coordinates[4][row]]
    L_F_Elbow = [coordinates[5][row], coordinates[6][row]]
    L_B_Paw = [coordinates[7][row], coordinates[8][row]]
    L_B_Wrist = [coordinates[9][row], coordinates[10][row]]
    L_B_Elbow = [coordinates[11][row], coordinates[12][row]]
    R_F_Paw = [coordinates[13][row], coordinates[14][row]]
    R_F_Wrist = [coordinates[15][row], coordinates[16][row]]
    R_F_Elbow = [coordinates[17][row], coordinates[18][row]]
    R_B_Paw = [coordinates[19][row], coordinates[20][row]]
    R_B_Wrist = [coordinates[21][row], coordinates[22][row]]
    R_B_Elbow = [coordinates[23][row], coordinates[24][row]]
    Tailset = [coordinates[25][row], coordinates[26][row]]
    Tailtip = [coordinates[27][row], coordinates[28][row]]
    L_Base_Ear = [coordinates[29][row], coordinates[30][row]]
    R_Base_Ear = [coordinates[31][row], coordinates[32][row]]
    Nose = [coordinates[33][row], coordinates[34][row]]
    Chin = [coordinates[35][row], coordinates[36][row]]
    L_Tip_Ear = [coordinates[37][row], coordinates[38][row]]
    R_Tip_Ear = [coordinates[39][row], coordinates[40][row]]
    L_Eye = [coordinates[41][row], coordinates[42][row]]
    R_Eye = [coordinates[43][row], coordinates[44][row]]
    Withers = [coordinates[45][row], coordinates[46][row]]
    Throat = [coordinates[47][row], coordinates[48][row]]
    #height = coordinates[49][row]
    #width = coordinates[50][row]

    # get a filled keypoint for the dog's back, front, up and down
    back = getFilledCoordinate([Tailset, L_B_Paw, R_B_Paw])
    front = getFilledCoordinate([Nose, Chin, L_Base_Ear, R_Base_Ear])
    up = getFilledCoordinate([L_Base_Ear, R_Base_Ear, L_Eye, R_Eye, Nose, Chin])
    down = getFilledCoordinate([L_F_Paw, R_F_Paw, L_B_Paw, R_F_Paw])

    # determine direction the dog is standing
    lookingLeft = None
    if back is not None and front is not None:
        if front[0] <= back[0]: lookingLeft = True
        else: lookingLeft = False

    # mirror image
    if lookingLeft is False:
        L_F_Paw = mirrorImage(width,L_F_Paw)
        L_F_Wrist = mirrorImage(width,L_F_Wrist)
        L_F_Elbow = mirrorImage(width,L_F_Elbow)
        L_B_Paw = mirrorImage(width,L_B_Paw)
        L_B_Wrist = mirrorImage(width,L_B_Wrist)
        L_B_Elbow = mirrorImage(width,L_B_Elbow)
        R_F_Paw = mirrorImage(width,R_F_Paw)
        R_F_Wrist = mirrorImage(width,R_F_Wrist)
        R_F_Elbow = mirrorImage(width,R_F_Elbow)
        R_B_Paw = mirrorImage(width,R_B_Paw)
        R_B_Wrist = mirrorImage(width,R_B_Wrist)
        R_B_Elbow = mirrorImage(width,R_B_Elbow)
        Tailset = mirrorImage(width, Tailset)
        Tailtip = mirrorImage(width,Tailtip)
        L_Base_Ear = mirrorImage(width,L_Base_Ear)
        R_Base_Ear = mirrorImage(width,R_Base_Ear)
        Nose = mirrorImage(width,Nose)
        Chin = mirrorImage(width,Chin)
        L_Tip_Ear = mirrorImage(width,L_Tip_Ear)
        R_Tip_Ear = mirrorImage(width,R_Tip_Ear)
        L_Eye = mirrorImage(width,L_Eye)
        R_Eye = mirrorImage(width,R_Eye)
        Withers = mirrorImage(width,Withers)
        Throat = mirrorImage(width,Throat)

    # scale image
    L_F_Paw = scaleImage(width, height, L_F_Paw)
    L_F_Wrist = scaleImage(width, height, L_F_Wrist)
    L_F_Elbow = scaleImage(width, height, L_F_Elbow)
    L_B_Paw = scaleImage(width, height, L_B_Paw)
    L_B_Wrist = scaleImage(width, height, L_B_Wrist)
    L_B_Elbow = scaleImage(width, height, L_B_Elbow)
    R_F_Paw = scaleImage(width, height, R_F_Paw)
    R_F_Wrist = scaleImage(width, height, R_F_Wrist)
    R_F_Elbow = scaleImage(width, height, R_F_Elbow)
    R_B_Paw = scaleImage(width, height, R_B_Paw)
    R_B_Wrist = scaleImage(width, height, R_B_Wrist)
    R_B_Elbow = scaleImage(width, height, R_B_Elbow)
    Tailset = scaleImage(width, height, Tailset)
    Tailtip = scaleImage(width, height, Tailtip)
    L_Base_Ear = scaleImage(width, height, L_Base_Ear)
    R_Base_Ear = scaleImage(width, height, R_Base_Ear)
    Nose = scaleImage(width, height, Nose)
    Chin = scaleImage(width, height, Chin)
    L_Tip_Ear = scaleImage(width, height, L_Tip_Ear)
    R_Tip_Ear = scaleImage(width, height, R_Tip_Ear)
    L_Eye = scaleImage(width, height, L_Eye)
    R_Eye = scaleImage(width, height, R_Eye)
    Withers = scaleImage(width, height, Withers)
    Throat = scaleImage(width, height, Throat)

    data.append([emotion,
                 L_F_Paw[0],
                 L_F_Paw[1],
                 L_F_Wrist[0],
                 L_F_Wrist[1],
                 L_F_Elbow[0],
                 L_F_Elbow[1],
                 L_B_Paw[0],
                 L_B_Paw[1],
                 L_B_Wrist[0],
                 L_B_Wrist[1],
                 L_B_Elbow[0],
                 L_B_Elbow[1],
                 R_F_Paw[0],
                 R_F_Paw[1],
                 R_F_Wrist[0],
                 R_F_Wrist[1],
                 R_F_Elbow[0],
                 R_F_Elbow[1],
                 R_B_Paw[0],
                 R_B_Paw[1],
                 R_B_Wrist[0],
                 R_B_Wrist[1],
                 R_B_Elbow[0],
                 R_B_Elbow[1],
                 Tailset[0],
                 Tailset[1],
                 Tailtip[0],
                 Tailtip[1],
                 L_Base_Ear[0],
                 L_Base_Ear[1],
                 R_Base_Ear[0],
                 R_Base_Ear[1],
                 Nose[0],
                 Nose[1],
                 Chin[0],
                 Chin[1],
                 L_Tip_Ear[0],
                 L_Tip_Ear[1],
                 R_Tip_Ear[0],
                 R_Tip_Ear[1],
                 L_Eye[0],
                 L_Eye[1],
                 R_Eye[0],
                 R_Eye[1],
                 Withers[0],
                 Withers[1],
                 Throat[0],
                 Throat[1]])

df = pd.DataFrame(data)
df.to_csv(outputPath, index=False, header=False)

###Pose Estimation

In [ ]:
sourcePath = '/content/drive/MyDrive/Image data/keypoints.csv'
outputPath = '/content/drive/MyDrive/Image data/poses.csv'

b_paw = 'get_fucked'
b_elbow = 'arsehole'

coordinates = pd.read_csv(sourcePath, header=None)

test = False
testNumber = 20

data = []
names = ['filepath',
         'bodyWeight',
         'tailPosition',
         'headPosition',
         'earPosition',
         'mouthCondition',
         'frontLeg',
         'backLeg']

#####################################################################
#####################################################################

# check if coordinate exists
def coordinateNotNull(tuple):
    if tuple == (-1,-1): return False
    return True

# get angle between 3 coordinates at coordinate b
def getAngle(a, b, c):
    ba = np.asarray(a)-np.asarray(b)
    bc = np.asarray(c)-np.asarray(b)
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(cosine_angle))

# determine line equation with 2 coordinates
def getLineEquation(p1,p2):
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
    intercept = p1[1] - slope * p1[0]
    return slope, intercept

# calculate if point is above a line determined by 2 coordinates
# (0 = point is on line)
def isRaised(lineStart, lineEnd, point):
    if lineStart[0] == lineEnd[0]: return 0
    slope, intercept = getLineEquation(lineStart, lineEnd)
    lineY = intercept + slope*point[0]
    if lineY >= point[1]:
        return 1
    else: return -1

# check if point is before checkpoint
# before is determined by the direction the dog faces
def isBefore(checkpoint, point, left):
    if left:
        if (checkpoint[0] <= point[0]): return True
        else: return False
    else:
        if (checkpoint[0] >= point[0]): return True
        else: return False

# check if dog faces the camera frontal
def isFrontal(back, front, up, down):
    width = abs(back[0] - front[0])
    height = abs(down[1] - up[1])
    if height/width > 4:
        return True
    return False

# get intersection of 2 lines determined by two coordinates each
# None = no intersection
def getIntersection(line1p1, line1p2, line2p1, line2p2):
    if line1p1[0] != line1p2[0] and line2p1[0] != line2p2[0]:
        slope1, intercept1 = getLineEquation(line1p1, line1p2)
        slope2, intercept2 = getLineEquation(line2p1, line2p2)
    else: return None
    if slope1 != slope2:
        x = (intercept1 - intercept2) / (slope2 - slope1)
        y = slope1 * x + intercept1
        return (x,y)
    return None

# get the first filled coordinate of a list
# None = no filled coordinate in list
def getFilledCoordinate(pointList):
    for point in pointList:
        if coordinateNotNull(point): return point
    return None

#####################################################################
#####################################################################

if test:
    imageNumber = testNumber
else: imageNumber = len(coordinates.index)

counter = 0
# iterate over images
for row in range(0,imageNumber):
    counter += 1
    newRow = []

    ###########################################################
    # create tuple for each dog keypoint
    L_F_Paw = (coordinates[1][row], coordinates[2][row])
    L_F_Wrist = (coordinates[3][row], coordinates[4][row])
    L_F_Elbow = (coordinates[5][row], coordinates[6][row])
    L_B_Paw = (coordinates[7][row], coordinates[8][row])
    L_B_Wrist = (coordinates[9][row], coordinates[10][row])
    L_B_Elbow = (coordinates[11][row], coordinates[12][row])
    R_F_Paw = (coordinates[13][row], coordinates[14][row])
    R_F_Wrist = (coordinates[15][row], coordinates[16][row])
    R_F_Elbow = (coordinates[17][row], coordinates[18][row])
    R_B_Paw = (coordinates[19][row], coordinates[20][row])
    R_B_Wrist = (coordinates[21][row], coordinates[22][row])
    R_B_Elbow = (coordinates[23][row], coordinates[24][row])
    Tailset = (coordinates[25][row], coordinates[26][row])
    Tailtip = (coordinates[27][row], coordinates[28][row])
    L_Base_Ear = (coordinates[29][row], coordinates[30][row])
    R_Base_Ear = (coordinates[31][row], coordinates[32][row])
    Nose = (coordinates[33][row], coordinates[34][row])
    Chin = (coordinates[35][row], coordinates[36][row])
    L_Tip_Ear = (coordinates[37][row], coordinates[38][row])
    R_Tip_Ear = (coordinates[39][row], coordinates[40][row])
    L_Eye = (coordinates[41][row], coordinates[42][row])
    R_Eye = (coordinates[43][row], coordinates[44][row])
    Withers = (coordinates[45][row], coordinates[46][row])
    Throat = (coordinates[47][row], coordinates[48][row])

    # get a filled keypoint for the dog's back, front, up and down
    back = getFilledCoordinate([Tailset, L_B_Paw, R_B_Paw])
    front = getFilledCoordinate([Nose, Chin, L_Base_Ear, R_Base_Ear])
    up = getFilledCoordinate([L_Base_Ear, R_Base_Ear, L_Eye, R_Eye, Nose, Chin])
    down = getFilledCoordinate([L_F_Paw, R_F_Paw, L_B_Paw, R_F_Paw])

    # determine existing neck point
    if coordinateNotNull(Withers): neck = Withers
    elif coordinateNotNull(Throat): neck = Throat
    else: neck = (-1, -1)

    # determine if dog is standing frontal
    frontal = False
    if back is not None and front is not None and up is not None and down is not None:
        frontal = isFrontal(back, front, up, down)

    # determine direction the dog is standing
    lookingLeft = None
    if back is not None and front is not None:
        if front[0] <= back[0]: lookingLeft = True
        else: lookingLeft = False

    # determine existing ear coordinates
    base_ear, tip_ear = (-1,-1), (-1,-1)
    if lookingLeft:
        if (coordinateNotNull(L_Base_Ear) and coordinateNotNull(L_Tip_Ear)): base_ear, tip_ear = L_Base_Ear, L_Tip_Ear
        elif (coordinateNotNull(R_Base_Ear) and coordinateNotNull(R_Tip_Ear)): base_ear, tip_ear = R_Base_Ear, R_Tip_Ear
    else:
        if (coordinateNotNull(R_Base_Ear) and coordinateNotNull(R_Tip_Ear)): base_ear, tip_ear = R_Base_Ear, R_Tip_Ear
        elif (coordinateNotNull(L_Base_Ear) and coordinateNotNull(L_Tip_Ear)): base_ear, tip_ear = L_Base_Ear, L_Tip_Ear

    ###########################################################
    # emotion
    newRow.append(coordinates[0][row])
    # emotion = coordinates[0][row]
    # if 'anger' in emotion: newRow.append(1)
    # else: newRow.append(0)
    # if 'fear' in emotion: newRow.append(1)
    # else: newRow.append(0)
    # if 'happiness' in emotion: newRow.append(1)
    # else: newRow.append(0)
    # if 'relaxation' in emotion: newRow.append(1)
    # else: newRow.append(0)

    # __________________________________________________________
    # calculate bodyweight distribution
    if coordinateNotNull(Tailset) \
        and coordinateNotNull(neck) \
        and lookingLeft is not None and not frontal \
        and ((coordinateNotNull(L_F_Paw) and coordinateNotNull(L_B_Paw))
             or (coordinateNotNull(R_F_Paw) and coordinateNotNull(R_B_Paw))):

        # determine existing paw pair
        if coordinateNotNull(L_F_Paw) and coordinateNotNull(L_B_Paw): paw1, paw2 = L_F_Paw, L_B_Paw
        else: paw1, paw2 = R_F_Paw, R_B_Paw
        intersection = getIntersection(paw1, paw2, neck, Tailset)
        if intersection is None: bodyweight = 0
        else:
            bodyweight = getAngle(neck, intersection, paw1)
            if isBefore(neck, intersection, lookingLeft): bodyweight = bodyweight*-1
            if neck == Throat: bodyweight = -14.1 + 0.35 * bodyweight
            newRow.append(bodyweight)
    else: newRow.append(None)

    # ___________________________________________________________
    # tail position
    if coordinateNotNull(neck) and coordinateNotNull(Tailset) and coordinateNotNull(Tailtip):
        angle = getAngle(Tailtip, Tailset, neck)
        raised = isRaised(neck, Tailset, Tailtip)
        if raised == 1: tailPosition = 180 - angle
        elif raised == -1: tailPosition = -1 * (180 - angle)
        else: tailPosition = None
        if tailPosition is not None and neck == Throat: tailPosition = 18.7 + 0.98*tailPosition
        newRow.append(tailPosition)
    else: newRow.append(None)

    # ___________________________________________________________
    # head position
    if coordinateNotNull(neck) and coordinateNotNull(Nose) \
        and lookingLeft is not None and not frontal \
        and ((coordinateNotNull(L_F_Paw) and coordinateNotNull(L_B_Paw)) or
            (coordinateNotNull(R_F_Paw) and coordinateNotNull(R_B_Paw))):

        if coordinateNotNull(L_F_Paw) and coordinateNotNull(L_B_Paw): paw1, paw2 = L_F_Paw, L_B_Paw
        else: paw1, paw2 = R_F_Paw, R_B_Paw
        intersection = getIntersection(paw1, paw2, neck, Nose)
        if intersection is None: headPosition = 0
        else:
            headPosition = getAngle(neck, intersection, paw1)
            if not isBefore(neck, intersection, lookingLeft): headPosition = -headPosition
            if neck == Throat: headPosition = -2.78 + 0.06*headPosition
        newRow.append(headPosition)
    else: newRow.append(None)

    # ___________________________________________________________
    # ear position
    if coordinateNotNull(Nose) and coordinateNotNull(base_ear) and coordinateNotNull(tip_ear):
        earPosition = getAngle(Nose, base_ear, tip_ear)
        raised = isRaised(Nose, base_ear, tip_ear)
        if raised == 1: earPosition = 180 - earPosition
        elif raised == -1: earPosition = -1 * (180 - earPosition)
        else: earPosition = None
        newRow.append(earPosition)
    else: newRow.append(None)

    # ___________________________________________________________
    # mouth condition
    if coordinateNotNull(Nose) and coordinateNotNull(base_ear) and coordinateNotNull(Chin):
        mouthCondition = getAngle(Nose, base_ear, Chin)
        newRow.append(mouthCondition)
    else: newRow.append(None)

    # ___________________________________________________________
    # front leg
    if lookingLeft:
        if coordinateNotNull(L_F_Paw) and coordinateNotNull(L_F_Elbow):
            f_paw, f_elbow = L_F_Paw, L_F_Elbow
        elif coordinateNotNull(R_F_Paw) and coordinateNotNull(R_F_Elbow):
            f_paw, f_elbow = R_F_Paw, R_F_Elbow
    else:
        if coordinateNotNull(R_F_Paw) and coordinateNotNull(R_F_Elbow):
            f_paw, f_elbow = R_F_Paw, R_F_Elbow
        elif coordinateNotNull(L_F_Paw) and coordinateNotNull(L_F_Elbow):
            f_paw, f_elbow = L_F_Paw, L_F_Elbow

    if coordinateNotNull(f_paw) and coordinateNotNull(f_elbow) and coordinateNotNull(neck):
        frontLegCondition = getAngle(f_paw, f_elbow, neck)
        if neck == Throat: 83.78 + 0.61*frontLegCondition
        newRow.append(frontLegCondition)
    else: newRow.append(None)

    # ___________________________________________________________
    # back leg
    if lookingLeft:
        if coordinateNotNull(L_B_Paw) and coordinateNotNull(L_B_Elbow):
            b_paw, b_elbow = L_B_Paw, L_B_Elbow
        elif coordinateNotNull(R_B_Paw) and coordinateNotNull(R_B_Elbow):
            b_paw, b_elbow = R_B_Paw, R_B_Elbow
    else:
        if coordinateNotNull(R_B_Paw) and coordinateNotNull(R_B_Elbow):
            b_paw = R_B_Paw
            b_elbow = R_B_Elbow
        elif coordinateNotNull(L_B_Paw) and coordinateNotNull(L_B_Elbow):
            b_paw, b_elbow = L_B_Paw, L_B_Elbow

    if coordinateNotNull(b_paw) and coordinateNotNull(b_elbow) and coordinateNotNull(Tailset):
        backLegCondition = getAngle(b_paw, b_elbow, Tailset)
        newRow.append(backLegCondition)
    else:
        newRow.append(None)

    data.append(newRow)

df = pd.DataFrame(data)
df.to_csv(outputPath, index=False, header=names)

### Classification with poses

#### Loading Data

In [ ]:
df = pd.read_csv('poses.csv')
emotion_dirty = df['filepath'].str.split('/')
emotion_clean = []
for e in emotion_dirty:
  filename = e[-1]
  emotion = filename.split('_')[0]
  emotion_clean.append(emotion)
df.insert(loc = 1,
                column = 'emotion',
                value = emotion_clean)
df.head(10)

In [ ]:
from sklearn.model_selection import train_test_split as split

training, test = split(df, test_size = 0.1, random_state = 42)
train, validation = split(training, test_size = 0.1, random_state = 42)

#### Building multi-input model

In [ ]:
model_name = 'resnet50_v01'

# load a new instance of the model.
base_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(224,224,3),
                                              include_top=True,
                                              weights='imagenet',
                                                    )

#model = tf.keras.applications.resnet50.ResNet50(input_shape=(224,224,3),
 #                                             include_top=True,
  #                                            weights='imagenet',
   #                                                 )
for layer in base_model.layers[-5:]:
  print(layer.name)

last_conv_layer = base_model.get_layer('conv5_block3_out')

conv_model = Model(inputs=base_model.input,
                   outputs=last_conv_layer.output)

new_model = Sequential()

new_model.add(conv_model)

new_model.add(Dropout(0.5))

new_model.add(tf.keras.layers.GlobalAveragePooling2D())


new_model.add(tf.keras.layers.Dense(4, activation='softmax'))

conv_model.trainable = True
for layer in conv_model.layers:
 # if(('conv5' not in layer.name) or ('block' not in layer.name) or ('bn' in layer.name)):
   if('bn' in layer.name):
    layer.trainable = False

new_model.summary()

#### Data Generator

In [ ]:
from sklearn.utils import shuffle
from PIL import Image, ImageEnhance

In [ ]:
def sample_df(df, col_name='emotion', n_sample_per_class=140, replace = False):
  samples = df.groupby(col_name)
  list_cls = df[col_name].unique()
  df_lst = []
  for cls in list_cls:
      cls_df = samples.get_group(cls)
      if (cls_df.shape[0] < n_sample_per_class) and (replace==False):
        cls_sample = cls_df
      else:
        cls_sample = cls_df.sample(n=n_sample_per_class,replace=replace,random_state=42)
      df_lst.append(cls_sample)
  df_sampled = pd.concat(df_lst, sort=False)
  df_sampled = shuffle(df_sampled)

  return df_sampled

In [ ]:
## Encoding for emtions
emotion_dict = {'Anger': 1,
                'Fear ': 2,
                'Happiness' : 3,
                'Relaxation' : 4}

In [ ]:
class GoodboiGenerator():
    """
   This is a data generator for the Goodboi dataset, adapted from the code used 
   in this article: 
   https://towardsdatascience.com/building-a-multi-output-convolutional-neural-network-with-keras-ed24c7bc1178
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self, random_state= 1313):
        p = np.random.RandomState(seed= random_state).permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        
        return train_idx, valid_idx, test_idx



#Now to preprocess and generate the images

    def preprocess_image(self, img_path, IMG_SIZE = 224, training = False):
        """
        Used to perform some minor preprocessing on the image before inputting 
        into the network.
        """
        im = tf.keras.preprocessing.image.load_img(img_path)
        im = im.resize((IMG_SIZE, IMG_SIZE))
        if training == True:
          img_contr_obj=ImageEnhance.Contrast(im)
          img_sharp_obj=ImageEnhance.Sharpness(im)
          factor=2.5
          im=img_contr_obj.enhance(factor)
          im = img_sharp_obj.enhance(factor)
          im = np.array(im).astype(np.float32) / 255.0
          im = tf.keras.preprocessing.image.random_brightness(im, 
                                                              brightness_range=(0.9, 1.1))
        else:
          im = np.array(im).astype(np.float32) / 255.0
        return im
        
    def generate_images(self,
                        image_idx,
                        is_training,
                        sampling,
                        image_aug = False,
                        batch_size=16,
                        IMG_SIZE = 224):
        """
        Used to generate a batch with images when training/testing/validating 
        our Keras model.
        """
        
        # arrays to store our batched data
        images = []
        bodyWeights = []
        tailPositions = []
        headPositions = []
        earPositions = []
        mouthConditions = []
        frontlegs = []
        backlegs = []
        emotions = []
        
        #This next section will upsample teh 
        if sampling:
            sample = self.df.iloc[image_idx]
            sampled = sample_df(sample, 
                    'emotion',
                    n_sample_per_class= int(sample['emotion'].value_counts().median()),
                    replace = True
                    )
            sampled.reset_index(inplace= True)
            image_idx = sampled.index
        else:
          sampled = self.df.iloc[image_idx]
          sampled.reset_index(inplace= True)
          image_idx =  sampled.index
        while True:
          
          for idx in image_idx:
                slice = sampled.iloc[idx]
                
                image = slice['filepath']
                bodyweight = slice['bodyWeight']
                tailposition = slice['tailPosition']
                headposition = slice['tailPosition']
                earposition = slice['earPosition']
                mouthcondition = slice['mouthCondition']
                frontleg = slice['frontLeg']
                backleg = slice['backLeg']
                target = slice['emotion']
                emotion = emotion_dict[target]
                
                im = self.preprocess_image(image,
                                           IMG_SIZE = IMG_SIZE,
                                           training = image_aug)

          # encoding targets
                images.append(im)
                bodyWeights.append(bodyweight)
                tailPositions.append(tailposition)
                headPositions.append(headposition)
                earPositions.append(earposition)
                mouthConditions.append(mouthcondition)
                frontlegs.append(frontleg)
                backlegs.append(backleg)
                emotions.append(emotion)
                
                # yielding condition
                if len(images) >= batch_size:
                    #emotions = tf.keras.utils.to_categorical(emotions, 4)
                    #yield [np.array(images),[np.array(bodyWeights), np.array(tailPositions), np.array(headPositions), np.array(earPositions), np.array(mouthConditions), np.array(frontlegs), np.array(backlegs)]] , np.array(emotions)
                    yield np.array(images) ,  np.array(emotions)
                    images = []
                    bodyWeights = []
                    tailPositions = []
                    headPositions = []
                    earPositions = []
                    mouthConditions = []
                    frontlegs = []
                    backlegs = []
                    emotions = []
                    
          if not is_training:
                break

In [ ]:
TRAIN_TEST_SPLIT = 0.8
data_generator = GoodboiGenerator(training)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes()
print(len(train_idx), len(valid_idx), len(test_idx))

In [ ]:
batch_size = 8
valid_batch_size = 8
train_gen = data_generator.generate_images(train_idx,
                                           is_training=True,
                                           sampling= True,
                                           image_aug= True,
                                           batch_size=batch_size,
                                           IMG_SIZE = 224)
valid_gen = data_generator.generate_images(valid_idx,
                                           is_training=True,
                                           sampling = False,
                                           batch_size=valid_batch_size,
                                           IMG_SIZE= 224)

In [ ]:
next(train_gen)

In [ ]:
playtest = next(train_gen)[0][0]


image_to_play = tf.keras.preprocessing.image.array_to_img(playtest)
plt.imshow(image_to_play)
plt.show()

#### Data Generator 2

In [ ]:
image_gen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=(.8,1.2),
    shear_range=0.005,
    zoom_range=[0.9, 1.4],
    channel_shift_range=0.0,
    fill_mode='nearest', cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    preprocessing_function=None,
    data_format=None,
    dtype=None
)

image_gen_val = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    dtype=None
)

In [ ]:
batch_size = 8
epochs = 100


In [ ]:
train_gen = image_gen_train.flow_from_dataframe(dataframe = train,
                                               directory = None,
                                               x_col = 'filepath',
                                               y_col = 'emotion',
                                               target_size= (224, 224),
                                               batch_size= batch_size,
                                               shuffle = True,
                                               class_mode = 'sparse')
val_gen = image_gen_train.flow_from_dataframe(dataframe = validation,
                                               directory = None,
                                               x_col = 'filepath',
                                               y_col = 'emotion',
                                               target_size= (224, 224),
                                               batch_size= 2,
                                              shuffle = True,
                                               class_mode = 'sparse')
test_gen = image_gen_val.flow_from_dataframe(dataframe = test,
                                              directory = None,
                                               x_col = 'filepath',
                                               y_col = 'emotion',
                                               target_size= (224, 224),
                                               batch_size= 1,
                                               class_mode = 'sparse')

In [ ]:
train_gen

#### Model Training

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weight = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(train_gen.classes),
                                    y=train_gen.classes)
class_weight
class_weight_dict = { i : class_weight[i] for i in range(0, len(class_weight) ) }
class_weight_dict

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("models/saved_models/resnet50_v5.h5", 
                             monitor='sparse_categorical_accuracy', 
                             verbose=1, save_best_only=True, 
                             save_weights_only=True, mode='auto', save_freq= 'epoch')

early = EarlyStopping(monitor='sparse_categorical_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0000000000001)

calls = [checkpoint, early, reduce_lr]

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
optimizer = Adam(learning_rate=1e-4)

loss = 'sparse_categorical_crossentropy'

lst_metrics = ['sparse_categorical_accuracy', recall_m, precision_m, f1_m]

new_model.compile(optimizer=optimizer, loss=loss, metrics=lst_metrics)

In [ ]:
history = new_model.fit(train_gen,
                    epochs=100,
                    steps_per_epoch= 70,
                    validation_data=val_gen,
                    class_weight=class_weight_dict,
                    validation_steps = 30,
                    callbacks = calls
                    ) 

In [ ]:
new_model.evaluate(test_gen)